In [41]:
import time
import datetime
import pytz
import requests
import json
from base64 import b64encode
import os
from dotenv import load_dotenv
import sys

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [31]:
# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variables
api_token_string = os.getenv('API_KEY')

# Check if the API key was loaded correctly
if api_token_string is None:
    raise ValueError("API_KEY environment variable not set")

api_token_string = api_token_string + ':api_token'    

In [32]:
TOGGL_API_URL = 'https://api.track.toggl.com/api/v9/me'

headers = {
            'content-type': 'application/json',
            'Authorization' : 'Basic %s' %  b64encode(api_token_string.encode('utf-8')).decode("ascii")
          }

In [ ]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]

def get_google_credentials():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
  return(creds)

creds = get_google_credentials()
creds

In [ ]:
unix_time_minus_one_hour = int(time.time()) - 3600
print(unix_time_minus_one_hour)

In [ ]:
response = requests.get(TOGGL_API_URL + f'/time_entries?since={unix_time_minus_one_hour}', headers=headers)

# Ensure the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Pretty-print the JSON response
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)
else:
    print(f"Request failed with status code {response.status_code}")

In [ ]:
# Extract the 'start' field
start_time_str = data[0]['start']

# Convert to a datetime object
start_time_utc = datetime.datetime.fromisoformat(start_time_str)

# Convert to local time
local_timezone = pytz.timezone('America/New_York')  # Replace with your local timezone
start_time_local = start_time_utc.astimezone(local_timezone)

print(f"Start time in local time: {start_time_local}")

In [ ]:
# Look for non-null stop time to determine whether the time tracker entry actually 
# has a stop time (if not, it's still in progress).
if data[0]['stop'] is not None:
    entry_index = 0
elif len(data) >=2 and data[1]['stop'] is not None:
    entry_index = 1
else:
    entry_index = -1
    print("No valid entry found!")
print(entry_index)

In [60]:
if entry_index < 0:
    print("Aborting execution...")
    sys.exit("Execution aborted due to invalid entry_index")

In [ ]:
start_time_str = data[entry_index]['start']
stop_time_str = data[entry_index]['stop']
description_str = data[entry_index]['description']

print(description_str)
print(start_time_str)
print(stop_time_str)

In [ ]:
event = {
  'summary': description_str,
  'colorId': '4',
  'description': 'This event generated automatically from the Waldis toggl gateway script.',
  'start': {
    'dateTime': start_time_str,
  },
  'end': {
    'dateTime': stop_time_str,
  },
}
event

In [ ]:
try:
    service = build("calendar", "v3", credentials=creds)

    event = service.events().insert(calendarId='primary', body=event).execute()
    print ('Event created: %s' % (event.get('htmlLink')))

except HttpError as error:
    print(f"An error occurred: {error}")
